# **Install and Import**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
import easyocr
reader = easyocr.Reader(['th','en'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 79.6 MB/s eta 0:00:00


In [ ]:
#---import---
import requests
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
from io import BytesIO
import os
import cv2
import numpy as np
from google.colab import files
from google.colab.patches import cv2_imshow
import openpyxl
import pandas as pd
import re
import Levenshtein

In [ ]:
# template
# image id card
id_card_path = "/content/drive/MyDrive/Project/template/Default_Id_Card.jpg"

# Open the local image file directly
id_card = Image.open(id_card_path)
id_card.save("Default_Id_Card.jpg")

# **Demo**

In [ ]:
def Reader(image_path1, image_path2, output_path):
    #alignment
    img1 = cv2.imread(image_path1, cv2.IMREAD_COLOR)
    img2 = cv2.imread(image_path2, cv2.IMREAD_COLOR)


    sift = cv2.SIFT_create()
    keypoints1, descriptors1 = sift.detectAndCompute(img1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(img2, None)

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.5 * n.distance:
            good_matches.append(m)

    src_points = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_points = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    transformation_matrix, _ = cv2.estimateAffinePartial2D(src_points, dst_points)

    transformation_matrix = np.vstack([transformation_matrix, [0, 0, 1]])

    aligned_image = cv2.warpPerspective(img1, transformation_matrix, (img2.shape[1], img2.shape[0]))

    cv2.imwrite(output_path + '.jpg', aligned_image)
    #OCR
    Ocr_Result = {}
    Coordinates_Only = {}
    bounds = reader.readtext(aligned_image)
    Ocr_Result = bounds
    Coordinates_Only = [bound[0] for bound in bounds]

    def calculate_overlap_area(box1, box2):
        x1_1, y1_1 = box1[0]
        x2_1, y2_1 = box1[2]
        x1_2, y1_2 = box2[0]
        x2_2, y2_2 = box2[2]

        area_box1 = (x2_1 - x1_1) * (y2_1 - y1_1)

        x_overlap = max(0, min(x2_1, x2_2) - max(x1_1, x1_2))
        y_overlap = max(0, min(y2_1, y2_2) - max(y1_1, y1_2))

        overlap_area = x_overlap * y_overlap

        if area_box1 != 0:
            overlap_percentage = (overlap_area / area_box1) * 100
        else:
            overlap_percentage = 0

        return overlap_percentage

    OCR_Result = Ocr_Result
    ocr_bbox = Coordinates_Only
    # พิกัดของ bounding box ที่กำหนด
                                  #topleft    topright  bottomright  bottomleft
    defined_name_bbox = [[77, 94], [670, 94], [670, 144], [77, 144]]
    defined_surname_bbox = [[77, 94], [670, 94], [670, 144], [77, 144]]
    defined_name_eng_bbox = [[294, 132], [558, 132], [558, 162], [294, 162]]
    defined_surname_eng_bbox =[[330, 166], [558, 166], [558, 196], [330, 196]]
    defined_birth_date_bbox =  [[220, 202], [558, 202], [558, 230], [220, 230]]
    defined_birth_date_eng_bbox =  [[220, 235], [558, 235], [558, 268], [220, 267]]
    defined_address1_bbox = [[120, 295], [540, 295], [540, 325], [120, 325]]
    defined_address2_bbox = [[77, 331], [540, 330], [540, 362], [80, 357]]

    # กำหนดเกณฑ์ของพื้นที่ทับซ้อนที่ต้องการ
    name_overlap_threshold = 50
    surname_overlap_threshold = 50
    name_eng_overlap_threshold = 50
    surname_eng_overlap_threshold = 50
    birth_date_overlap_threshold = 50
    birth_date_eng_overlap_threshold = 50
    address1_overlap_threshold = 50
    address2_overlap_threshold = 50

    max_name_overlap_area = 0
    max_name_overlap_index = None
    name_values = []
    name_texts = []

    max_surname_overlap_area = 0
    max_surname_overlap_index = None
    surname_values = []
    surname_texts = []

    max_name_eng_overlap_area = 0
    max_name_eng_overlap_index = None
    nameeng_values = []
    nameeng_texts = []

    max_surname_eng_overlap_area = 0
    max_surname_eng_overlap_index = None
    surnameeng_values = []
    surnameeng_texts = []

    max_birth_date_overlap_area = 0
    max_birth_date_overlap_index = None
    birthday_values = []
    birthday_texts = []

    max_birth_date_eng_overlap_area = 0
    max_birth_date_eng_overlap_index = None
    birthdayeng_values = []
    birthdayeng_texts = []

    max_address1_overlap_area = 0
    max_address1_overlap_index = None
    address1_values = []
    address1_texts= []

    max_address2_overlap_area = 0
    max_address2_overlap_index = None
    address2_values = []
    address2_texts= []

    for i, ocr_box in enumerate(ocr_bbox):
          name_overlap_area = calculate_overlap_area(ocr_box, defined_name_bbox)
          surname_overlap_area = calculate_overlap_area(ocr_box, defined_surname_bbox)
          name_eng_overlap_area =calculate_overlap_area(ocr_box, defined_name_eng_bbox)
          surname_eng_overlap_area = calculate_overlap_area(ocr_box, defined_surname_eng_bbox)
          birth_date_overlap_area = calculate_overlap_area(ocr_box, defined_birth_date_bbox)
          birth_date_eng_overlap_area = calculate_overlap_area(ocr_box, defined_birth_date_eng_bbox)
          address1_overlap_area = calculate_overlap_area(ocr_box, defined_address1_bbox)
          address2_overlap_area = calculate_overlap_area(ocr_box, defined_address2_bbox)
          if name_overlap_area > name_overlap_threshold:
              max_name_overlap_area = name_overlap_area
              max_name_overlap_index = i
              name_values.append(OCR_Result[i][0][1][0])
              name_texts.append(OCR_Result[i][1])

          if surname_overlap_area > surname_overlap_threshold:
                max_surname_overlap_area = surname_overlap_area
                max_surname_overlap_index = i
                surname_values.append(OCR_Result[i][0][1][0])
                surname_texts.append(OCR_Result[i][1])

          if name_eng_overlap_area > name_eng_overlap_threshold:
                max_name_eng_overlap_area = name_eng_overlap_area
                max_name_eng_overlap_index = i
                nameeng_values.append(OCR_Result[i][0][1][0])
                nameeng_texts.append(OCR_Result[i][1])

          if surname_eng_overlap_area > surname_eng_overlap_threshold:
                max_surname_eng_overlap_area = surname_eng_overlap_area
                max_surname_eng_overlap_index = i
                surnameeng_values.append(OCR_Result[i][0][1][0])
                surnameeng_texts.append(OCR_Result[i][1])

          if birth_date_overlap_area > birth_date_overlap_threshold:
                max_birth_date_overlap_area = birth_date_overlap_area
                max_birth_date_overlap_index = i
                birthday_values.append(OCR_Result[i][0][1][0])
                birthday_texts.append(OCR_Result[i][1])

          if birth_date_eng_overlap_area > birth_date_eng_overlap_threshold:
                max_birth_date_eng_overlap_area = birth_date_eng_overlap_area
                max_birth_date_eng_overlap_index = i
                birthdayeng_values.append(OCR_Result[i][0][1][0])
                birthdayeng_texts.append(OCR_Result[i][1])

          if address1_overlap_area > address1_overlap_threshold:
                max_address1_overlap_area = address1_overlap_area
                max_address1_overlap_index = i
                address1_values.append(OCR_Result[i][0][1][0])
                address1_texts.append(OCR_Result[i][1])

          if address2_overlap_area > address2_overlap_threshold:
                max_address2_overlap_area = address2_overlap_area
                max_address2_overlap_index = i
                address2_values.append(OCR_Result[i][0][1][0])
                address2_texts.append(OCR_Result[i][1])

    #Name
    if name_values:
          sorted_name_texts = [text for _, text in sorted(zip(name_values, name_texts))]
          name = ' '.join(sorted_name_texts)
    else:
          name = ""
    #Surname
    if surname_values:
          sorted_surname_texts = [text for _, text in sorted(zip(surname_values, surname_texts))]
          surname = ' '.join(sorted_surname_texts)
    else:
          surname = ""
    #NameEng
    if nameeng_values:
          sorted_nameeng_texts = [text for _, text in sorted(zip(nameeng_values, nameeng_texts))]
          nameeng = ' '.join(sorted_nameeng_texts)
    else:
          nameeng = ""
    #SurnameEng
    if surnameeng_values:
          sorted_surnameeng_texts = [text for _, text in sorted(zip(surnameeng_values, surnameeng_texts))]
          surnameeng = ' '.join(sorted_surnameeng_texts)
    else:
          surnameeng = ""
    #Birthday
    if birthday_values:
          sorted_birthday_texts = [text for _, text in sorted(zip(birthday_values, birthday_texts))]
          birthday = ' '.join(sorted_birthday_texts)
    else:
          birthday = ""
    #BirthdayEng
    if birthdayeng_values:
          sorted_birthdayeng_texts = [text for _, text in sorted(zip(birthdayeng_values, birthdayeng_texts))]
          birthdayeng = ' '.join(sorted_birthdayeng_texts)
    else:
          birthdayeng = ""
    #Address1
    if address1_values:
          sorted_address1_texts = [text for _, text in sorted(zip(address1_values, address1_texts))]
          address1 = ' '.join(sorted_address1_texts)
    else:
          address1 = ""
    #Address2
    if address2_values:
          sorted_address2_texts = [text for _, text in sorted(zip(address2_values, address2_texts))]
          address2 = ' '.join(sorted_address2_texts)
    else:
          address2 = ""
    full_address = address1 + ' ' + address2

    def capitalize_and_handle_empty(cell_value):
          if pd.notna(cell_value):
              return ' '.join([word.capitalize() for word in str(cell_value).split()])
          return cell_value

    nameeng = capitalize_and_handle_empty(nameeng)
    surnameeng = capitalize_and_handle_empty(surnameeng)
    birthdayeng = capitalize_and_handle_empty(birthdayeng)
#-------------------------------------------------------------------------------------------------------
    pattern_to_delete = r"(เก|ว|ท)"
    pattern_to_keep = r'\b[ก-ฮ][ก-๙\d]+\d\b'

    words = birthday.split()
    words = [word for word in words if not (re.search(pattern_to_delete, word, flags=re.IGNORECASE) and not re.search(pattern_to_keep, word, flags=re.IGNORECASE))]
    birthday = ' '.join(words)
#-------------------------------------------------------------------------------------------------------
    def modify_prefix(cell_value):
          words = cell_value.split()
          for i in range(len(words)):
              for j in range(31, 0, -1):
                  pattern = re.compile(rf'\b[ก-ฮ][ก-๙\d]+{j}\b', re.IGNORECASE)
                  words[i] = pattern.sub(f'{j}', words[i])

          # Join the updated words back into a string
          birthday = ' '.join(words)
          return birthday

    birthday = modify_prefix(birthday)
#-------------------------------------------------------------------------------------------------------
    pattern_to_delete = r'(of|i|da|bt|h|at|af|bu|al|ol|ot|{|arn)'
    pattern_to_keep = r'\b[A-Za-z][A-Za-z\d]+\d\b'

    words = birthdayeng.split()
    words = [word for word in words if not (re.search(pattern_to_delete, word, flags=re.IGNORECASE) and not re.search(pattern_to_keep, word, flags=re.IGNORECASE))]
    birthdayeng = ' '.join(words)
#-------------------------------------------------------------------------------------------------------
    def modify_prefixeng(cell_value):
          words = cell_value.split()
          for i in range(len(words)):
              for j in range(31, 0, -1):
                  pattern = re.compile(rf'\b[A-za-z][A-za-z\d]+{j}\b', re.IGNORECASE)
                  words[i] = pattern.sub(f'{j}', words[i])

          birthdayeng = ' '.join(words)
          return birthdayeng

    birthdayeng = modify_prefixeng(birthdayeng)
#-------------------------------------------------------------------------------------------------------
    pattern_to_startwithB = r'\b[bB]\w*\b'
    words = birthdayeng.split()
    new_words = [word for word in words if not re.search(pattern_to_startwithB, word, flags=re.IGNORECASE)]
    birthdayeng = ' '.join(new_words)
#-------------------------------------------------------------------------------------------------------
    date_pattern = r'(ม.ค.|ก.พ.|มี.ค.|เม.ย.|พ.ค.|มิ.ย.|ก.ค.|ส.ค.|ก.ย.|ต.ค.|พ.ย.|ธ.ค.)'
    matches = re.findall(date_pattern, birthday)
    if matches:
        left_column = birthdayeng
        if 'ม.ค.' in birthday:
            birthdayeng = re.sub(r'J[A-Za-z][A-Za-z][.]|[A-Za-z]a[A-Za-z][.]|[A-ZA-Za-z][A-Za-z]n[.]', 'Jan.', left_column)
        if 'ก.พ.' in birthday:
            birthdayeng = re.sub(r'F[A-Za-z][A-Za-z][.]|[A-Za-z]e[A-Za-z][.]|[A-ZA-Za-z][A-Za-z]b[.]', 'Feb.', left_column)
        if 'มี.ค.' in birthday:
            birthdayeng = re.sub(r'M[A-Za-z][A-Za-z][.]|[A-Za-z]a[A-Za-z][.]|[A-ZA-Za-z][A-Za-z]r[.]', 'Mar.', left_column)
        if 'เม.ย.' in birthday:
            birthdayeng = re.sub(r'A[A-Za-z][A-Za-z][.]|[A-Za-z]p[A-Za-z][.]|[A-ZA-Za-z][A-Za-z]l[.]', 'Apr.', left_column)
        if 'พ.ค.' in birthday:
            birthdayeng = re.sub(r'M[A-Za-z][A-Za-z][.]|[A-Za-z]a[A-Za-z][.]|[A-ZA-Za-z][A-Za-z]y[.]', 'May.', left_column)
        if 'มิ.ย.' in birthday:
            birthdayeng = re.sub(r'J[A-Za-z][A-Za-z][.]|[A-Za-z]u[A-Za-z][.]|[A-ZA-Za-z][A-Za-z]n[.]', 'Jun.', left_column)
        if 'ก.ค.' in birthday:
            birthdayeng = re.sub(r'J[A-Za-z][A-Za-z][.]|[A-Za-z]u[A-Za-z][.]|[A-ZA-Za-z][A-Za-z]y[.]', 'Jul.', left_column)
        if 'ส.ค.' in birthday:
            birthdayeng = re.sub(r'A[A-Za-z][A-Za-z][.]|[A-Za-z]u[A-Za-z][.]|[A-ZA-Za-z][A-Za-z]g[.]', 'Aug.', left_column)
        if 'ก.ย.' in birthday:
            birthdayeng = re.sub(r'S[A-Za-z][A-Za-z][.]|[A-Za-z]e[A-Za-z][.]|[A-ZA-Za-z][A-Za-z]p[.]', 'Sep.', left_column)
        if 'ต.ค.' in birthday:
            birthdayeng = re.sub(r'0[A-Za-z][A-Za-z][.]|O[A-Za-z][A-Za-z][.]|[A-Za-z]c[A-Za-z][.]|[A-ZA-Za-z][A-Za-z]t[.]', 'Oct.', left_column)
        if 'พ.ย.' in birthday:
            birthdayeng = re.sub(r'N[A-Za-z][A-Za-z][.]|[A-Za-z]o[A-Za-z][.]|[A-ZA-Za-z][A-Za-z]v[.]', 'Nov.', left_column)
        if 'ธ.ค.' in birthday:
            birthdayeng = re.sub(r'D[A-Za-z][A-Za-z][.]|[A-Za-z]e[A-Za-z][.]|[A-ZA-Za-z][A-Za-z]c[.]', 'Dec.', left_column)
#-------------------------------------------------------------------------------------------------------
    month_patterns = {
        'Jan.': r'(ม|ค)',
        'Feb.': r'(ก|พ)',
        'Mar.': r'(มี|ค)',
        'Apr.': r'(เม|ย)',
        'May.': r'(พ|ค)',
        'Jun.': r'(มิ|ย)',
        'Jul.': r'(ก|ค)',
        'Aug.': r'(ส|ค)',
        'Sep.': r'(ก|ย)',
        'Oct.': r'(ต|ค)',
        'Nov.': r'(พ|ย)',
        'Dec.': r'(ธ|ค)'
    }
    date_pattern = r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)'
    matches = re.findall(date_pattern, birthdayeng)
    if matches:
        for month, pattern in month_patterns.items():
            if month in birthdayeng:
                words = birthday.split()
                words = [f'{"ม.ค." if month == "Jan." else "ก.พ." if month == "Feb." else "มี.ค." if month == "Mar." else "เม.ย." if month == "Apr." else "พ.ค." if month == "May." else "มิ.ย." if month == "Jun." else "ก.ค." if month == "Jul." else "ส.ค." if month == "Aug." else "ก.ย." if month == "Sep." else "ต.ค." if month == "Oct." else "พ.ย." if month == "Nov." else "ธ.ค."}' if re.search(pattern, word, flags=re.IGNORECASE) else word for word in words]
                birthday = ' '.join(words)
#-------------------------------------------------------------------------------------------------------
    def Name_prefix(cell_value):
          words = cell_value.split()
          patterns = [
              (re.compile(r'\bนาย\b', re.IGNORECASE), "นาย"),
              (re.compile(r'\bนาง\b', re.IGNORECASE), "นาง"),
              (re.compile(r'\bน.ส.\b', re.IGNORECASE), "น.ส."),
          ]

          for i in range(len(words)):
              for pattern, replacement in patterns:
                  if pattern.search(words[i]):
                      words[i] = replacement
          Prefix = ' '.join(words)
          return Prefix

    name = Name_prefix(name)
    surname = Name_prefix(surname)
#-------------------------------------------------------------------------------------------------------
    pattern_to_delete = r'([A-Za-z])'
    words = name.split()
    words = [word for word in words if not (re.search(pattern_to_delete, word, flags=re.IGNORECASE))]
    name = ' '.join(words)

    words = surname.split()
    words = [word for word in words if not (re.search(pattern_to_delete, word, flags=re.IGNORECASE))]
    surname = ' '.join(words)
#-------------------------------------------------------------------------------------------------------
    def parse_name(full_name):
          name_parts = full_name.split()
          if len(name_parts) == 2:
              if name_parts[0] in ['นาง', 'นาย', 'น.ส.']:
                return full_name, full_name
              else:
                first_name = name_parts[0]
                last_name = name_parts[1]
                return first_name, last_name
          elif len(name_parts) == 3:
              first_name = name_parts[0] + ' ' + name_parts[1]
              last_name = ' '.join(name_parts[2:])
              return first_name, last_name
          elif len(name_parts) == 4:
              first_name = name_parts[1] + ' ' + name_parts[2]
              last_name = ' '.join(name_parts[3:])
              return first_name, last_name
          elif len(name_parts) == 5:
              first_name = name_parts[2] + ' ' + name_parts[3]
              last_name = ' '.join(name_parts[4:])
              return first_name, last_name
          else:
              return full_name, full_name

    equality_check = name == surname
    if equality_check == True:
        first_name, last_name = parse_name(name)
        name = first_name
        surname = last_name
#-------------------------------------------------------------------------------------------------------
    def Nameeng_prefix(cell_value):
          words = cell_value.split()
          patterns = [
              (re.compile(r'\bMr\b', re.IGNORECASE), "Mr."),
              (re.compile(r'\bMrs\b', re.IGNORECASE), "Mrs."),
              (re.compile(r'\bMiss\b', re.IGNORECASE), "Miss"),
          ]
          for i in range(len(words)):
              for pattern, replacement in patterns:
                  if pattern.search(words[i]):
                      words[i] = replacement
          Prefix = ' '.join(words)
          return Prefix

    nameeng = Nameeng_prefix(nameeng)
#-------------------------------------------------------------------------------------------------------
    def convert(cell_value):
          words = cell_value.split()
          patterns = [
              (re.compile(r'\bl\b', re.IGNORECASE), "1")
          ]
          for i in range(len(words)):
              for pattern, replacement in patterns:
                  if pattern.search(words[i]):
                      words[i] = replacement
          updated_cell_value = ' '.join(words)
          return updated_cell_value

    full_address = convert(str(full_address))
#-------------------------------------------------------------------------------------------------------
    def spelling_v1(cell_value):
          words = cell_value.split()
          patterns = [
              (re.compile(r'ที่$|ที$|ท$|ท[า-ูเ-โ-ใ-ไ][์็ุูึเแโใไ]?$|ทฺ$|ทํ$|ทํฺ$', re.IGNORECASE), "หมู่ที่")
          ]
          for i in range(len(words)):
              for pattern, replacement in patterns:
                  if pattern.search(words[i]):
                      words[i] = replacement
          updated_cell_value = ' '.join(words)
          return updated_cell_value

    full_address = spelling_v1(str(full_address))
#-------------------------------------------------------------------------------------------------------
    def spelling_v2(cell_value):
          words = cell_value.split()
          for i in range(len(words)):
              for j in range(100, 0, -1):
                  first_pattern = re.compile(rf'[ก-๙][ก-๙][ก-๙]ที่{j}$|[ก-๙][ก-๙][ก-๙]ที{j}$|[ก-๙][ก-๙][ก-๙]ท{j}$|\b[ก-๙][ก-๙][ก-๙]ท[า-ูเ-โ-ใ-ไ][์็ุูึเแโใไ]*{j}\b|\b[ก-๙][ก-๙][ก-๙]ทฺ*{j}\b|\b[ก-๙][ก-๙][ก-๙]ทํ*{j}\b|\b[ก-๙][ก-๙][ก-๙]ทํฺ*{j}\b', re.IGNORECASE)
                  words[i] = first_pattern.sub(f'หมู่ที่ {j}', words[i])
                  second_pattern = re.compile(rf'\b[ก-๙][ก-๙]ที่+{j}$\b|\b[ก-๙][ก-๙]ที+{j}$\b|\b[ก-๙][ก-๙]ท+{j}$\b|\b[ก-๙][ก-๙]ท[า-ูเ-โ-ใ-ไ][์็ุูึเแโใไ]*{j}\b|\b[ก-๙][ก-๙]ทฺ*{j}\b|\b[ก-๙][ก-๙]ทํ*{j}\b|\b[ก-๙][ก-๙]ทํฺ*{j}\b', re.IGNORECASE)
                  words[i] = second_pattern.sub(f'หมู่ที่ {j}', words[i])
          updated_cell_value = ' '.join(words)
          return updated_cell_value

    full_address = spelling_v2(str(full_address))
#-------------------------------------------------------------------------------------------------------
    patterns = ["ต.", "อ.", ".", "ด.", "ฮ."]
    def delete_aloneprefix(text):
        if isinstance(text, str):
            words = text.split()
            filtered_words = [word for word in words if word not in patterns]
            return ' '.join(filtered_words)
        return text

    full_address = delete_aloneprefix(full_address)
#-------------------------------------------------------------------------------------------------------
    def text_edit_distance(s1, s2):
          return Levenshtein.distance(s1, s2)

    min_distances = {}
    file2 = pd.read_excel('/content/database_address.xlsx')
    for index2, row_dataset in file2.iterrows():
          address1_words = str(full_address).split()
          for word in address1_words:
                if word.isdigit() or word == 'หมู่ที่' or re.match(r"\bถ\.[\wะ-์]+\b", word) or re.match(r"\bซ\.[\wะ-์]+\b", word):
                    continue
                distance_address1 = text_edit_distance(word, str(row_dataset['data_address']))
                if distance_address1 <= 3 and (word not in min_distances or distance_address1 < min_distances[word]['distance']):
                    min_distances[word] = {'distance': distance_address1, 'similaraddress': str(row_dataset['data_address'])}

    address1_words = str(full_address).split()
    for i, word in enumerate(address1_words):
        if word.isdigit() or word == 'หมู่ที่' or re.match(r"\bถ\.[\wะ-์]+\b", word) or re.match(r"\bซ\.[\wะ-์]+\b", word):
            continue
        if word in min_distances:
            best_match = min_distances[word]['similaraddress']
            address1_words[i] = best_match
    full_address = ' '.join(address1_words)
#-------------------------------------------------------------------------------------------------------

    print(name)
    print(surname)
    print(nameeng)
    print(surnameeng)
    print(birthday)
    print(birthdayeng)
    print(full_address)
    data = {
        'Name': [name],
        'Surname': [surname],
        'NameEng': [nameeng],
        'SurnameEng': [surnameeng],
        'Birthday': [birthday],
        'BirthdayEng': [birthdayeng],
        'FullAddress': [full_address]
    }
    df = pd.DataFrame(data)

    # Save the DataFrame to an Excel file
    output_file_name = f'{os.path.splitext(os.path.basename(image_path1))[0]}_output.xlsx'
    excel_output_path = os.path.join(output_root_directory, output_file_name)
    df.to_excel(excel_output_path, index=False)

    print("Data saved to Excel file:", excel_output_path)

    return



# **Run test**

In [ ]:
file_name = '/content/1.jpg'
output_root_directory = '/content/'
if file_name.endswith('.jpg') or file_name.endswith('.png'):
    # ใช้ os.path.splitext() เพื่อแยกชื่อไฟล์และนามสกุลไฟล์
    file_name_without_extension, _ = os.path.splitext(file_name)
    image_path1 = file_name
    image_path2 = "Default_Id_Card.jpg"

    # สร้างชื่อไฟล์เอาไว้สำหรับบันทึกรูปภาพที่ปรับแต่ง
    output_path = os.path.join(output_root_directory, file_name_without_extension)

    # เรียกใช้ฟังก์ชัน align_images เพื่อปรับแต่งรูปภาพ
    Reader(image_path1, image_path2, output_path)

นาย ธัญพิสิฐ
วชิรขจรชัย
Mr. Thunphisit
Wachirakajonchai
26 ก.พ. 2545
26 Feb. 2002
ที่อยู่ 138 ถ.พลับพลาไชย แขวงวัดเทพศิรินทร์ เขตป้อมปราบศัตรูพ่าย กรุงเทพมหานคร


In [ ]:
output_root_directory = '/content/'

for i in range(1, 16):
    file_name = f'/content/{i}.jpg'
    if os.path.exists(file_name):
        if file_name.endswith('.jpg') or file_name.endswith('.png'):
            # ใช้ os.path.splitext() เพื่อแยกชื่อไฟล์และนามสกุลไฟล์
            file_name_without_extension, _ = os.path.splitext(file_name)
            image_path1 = file_name
            image_path2 = "Default_Id_Card.jpg"

            # เรียกใช้ฟังก์ชัน align_images เพื่อปรับแต่งรูปภาพ
            Reader(image_path1, image_path2, output_root_directory)

ชื่อตัวและชื่อสกุล ฺนาย ธัญพิสิฐ วชิรขจรชัย
ชื่อตัวและชื่อสกุล ฺนาย ธัญพิสิฐ วชิรขจรชัย
mr. thunphisit
 wachirakajonchai
 เกิดวันที่ 26 ก.พ. 2545
 date of birth  26 feb.  2002
ที่อยู่ 138 ถ.พลับพลาไชย แขวงวัดเทพศิรินทร์ เขตป้อมปราบศัตรูพ่าย กรุงเทพมหานคร
Data saved to Excel file: /content/1_output.xlsx
ชื่อตัวและชื่อสกุล ฺนาย ธัญพิสิฐ วชิระเจรชัย
ชื่อตัวและชื่อสกุล ฺนาย ธัญพิสิฐ วชิระเจรชัย
 mr.  thunpaist
 wachirarajonchai
 เกิดวันที่ 26 ก.พ. 2545
 date of birth  26 feb.  2002
138 ถ.พลับพลาไชย แขวงวัดเทพศิรินทร์ เขตป้อมปราบศัตรูพ่าย กรุงเทพมหานคร
Data saved to Excel file: /content/2_output.xlsx
เชื่อตัวและชื่อสกุล นาย ธัญพิสิฐ วชิรขจรชัย
เชื่อตัวและชื่อสกุล นาย ธัญพิสิฐ วชิรขจรชัย
mr. thunphisit
 wachirakajonchai
 เกิดวันที่ 26 ก.พ. 2545
 date of birth  26 feb.  2002
ที่อยู่ 138 ถ.พลับพลาไชย แขวงวัดเทพศิรินทร์ เขตป้อมปราบศัตรูพ่าย กรุงเทพมหานคร
Data saved to Excel file: /content/3_output.xlsx
ชื่อตัวและชื่อสกุสฺนาย ธัญพิสิฐ วชิรขจรชัย
ชื่อตัวและชื่อสกุสฺนาย ธัญพิสิฐ วชิรขจรชัย
 mr. th

# **Upload**

In [ ]:
from IPython.display import display
import ipywidgets as widgets
from google.colab import files

output_root_directory = '/content/'
# อัปโหลดไฟล์รูปภาพ
uploaded_file = files.upload()

# หากมีไฟล์รูปภาพที่อัปโหลด เลือกไฟล์แรกมาใช้
if len(uploaded_file) > 0:
    file_name = list(uploaded_file.keys())[0]
    image_path1 = file_name
    image_path2 = "Default_Id_Card.jpg"

    # สร้างชื่อไฟล์เอาไว้สำหรับบันทึกรูปภาพที่ปรับแต่ง
    file_name_without_extension, _ = os.path.splitext(file_name)
    output_path = os.path.join(output_root_directory, file_name_without_extension)

    # เรียกใช้ฟังก์ชัน align_images เพื่อปรับแต่งรูปภาพ
    Reader(image_path1, image_path2, output_path)
else:
    print("ไม่มีไฟล์รูปภาพที่อัปโหลด")


Saving 9.jpg to 9.jpg
ชื่อตัวและชื่อสกุล ฺน.ส.ฺสพรรษา
สุวรรณอินทรีย์
Miss Suphansa
Suwaninsee
25 ก.ค. 2545
25 Jul. 2002
149 ซ.ร่มเกล้า 42 แขวงคลองสามประเวศ เขตลาดกระบัง กรุงเทพมหานคร
Data saved to Excel file: /content/9_output.xlsx
